In [ ]:
import pandas as pd
from datetime import datetime, date  # Added this import
from numpy import log, sqrt, exp
from scipy.stats import norm
from scipy.stats import norm
from scipy.optimize import newton
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import os


def plot_option_strategy_plotly(legs, spot_price, contracts=1):
    """
    Interactive Plotly payoff plot for multi-leg option strategy.

    Parameters:
    - legs: list of dicts, each with keys:
        - 'type': 'call' or 'put'
        - 'strike': strike price
        - 'position': 'long' or 'short'
        - 'premium': cost of option
    - spot_price: current price of the stock
    - contracts: multiplier (1 contract = 100 shares)
    """
    S = np.linspace(0.5 * spot_price, 1.5 * spot_price, 500)
    total_payoff = np.zeros_like(S)

    # Add each leg's payoff
    for leg in legs:
        payoff = option_leg_payoff(
            S,
            K=leg['strike'],
            option_type=leg['type'],
            position=leg['position'],
            premium=leg['premium'],
            stock_buying_price=leg['stock_buying_price'],
            derivative_type=leg['derivative_type']
        )
        total_payoff += contracts * 100 * payoff

    # Create interactive plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=S, y=total_payoff, mode='lines', name='Total Payoff', line=dict(width=3)))
    
    fig.add_shape(type="line", x0=S[0], x1=S[-1], y0=0, y1=0,
                  line=dict(color="black", width=1, dash="dash"), name='Breakeven')

    fig.add_vline(x=spot_price, line=dict(color='green', dash='dot'), annotation_text="Spot Price")

    fig.update_layout(
        title='Interactive Option Strategy Payoff',
        xaxis_title='Stock Price at Expiration',
        yaxis_title='Profit / Loss ($)',
        template='plotly_white',
        hovermode='x',
        showlegend=True
    )

    fig.show()

# plot payoff cureve for any option stategy above

# sample usage:
# covered call
# plot_option_strategy_plotly(
#     legs=[
#         {'type': 'call', 'strike': 200, 'position': 'short', 'premium': 0.66, stock_buying_price:0, derivative_type:'option'},
#         {'type': 'call', 'strike': 0, 'position': 'long', 'premium': 0, stock_buying_price:195, derivative_type:'stock'} # Simulates long stock
#     ],
#     spot_price=195,
#     contracts=1
# )

# butterfly option strategy:
# plot_option_strategy_plotly(
#     legs=[
#         {'type': 'call', 'strike': 180, 'position': 'long', 'premium': 8.50, 'stock_buying_price':0, 'derivative_type':'option'},
#         {'type': 'call', 'strike': 190, 'position': 'short', 'premium': 4.25, 'stock_buying_price':0, 'derivative_type':'option'},
#         {'type': 'call', 'strike': 190, 'position': 'short', 'premium': 4.25, 'stock_buying_price':0, 'derivative_type':'option'},
#         {'type': 'call', 'strike': 200, 'position': 'long', 'premium': 1.20, 'stock_buying_price':0, 'derivative_type':'option'},
#     ],
#     spot_price=190
# )